In [1]:
import numpy as np

In [2]:
def post_money_safe(val, inv):
    return inv / val * 100

def pre_money_safe(pre, stake, inv):
    return stake * pre / (pre+inv)

In [3]:
# B brings in 10MM on a 100MM post money valuation on ABC
# how much B owns?

post_val = 100
new_inv = 10

b_post_stake = post_money_safe(post_val, new_inv)
b_post_stake

10.0

In [5]:
# Investor A had 10% stake on ABC (10MM investment with a 100MM pre money Valuation)
# B brings in an additional 10MM on a new round
# how much A still have?

a_pre_stake = 50
pre_val = 90

a_post_stake = pre_money_safe(pre_val, a_pre_stake, new_inv)
a_post_stake

45.0

In [70]:
class Investment:

    def __init__(self, company, invested, round, covenant=None):
        self.round = round
        self.invested = invested
        self.price = company.price
        self.shares = self.invested / self.price
        self.covenant = covenant

class Investor:

    def __init__(self, name):
        self.name = name
        self.investments = []

    def new_investment(self, company, invested, round, covenant=None):
        investment = Investment(company, invested, round, covenant)
        self.investments.append(investment)

    def total_shares(self):
        shares = 0
        for i in self.investments:
            shares += i.shares 
        return shares

class Company:
    
    def __init__(self, name, equity, shares):
        self.name = name
        self.equity = equity
        self.shares = shares
        self.price = equity/shares
        self.round = 0
        self.founder = Investor('founder')
        self.founder.new_investment(self, self.equity, self.round)
        self.cap_table = [self.founder]

    def current_stake(self):
        cap = []
        for i in self.cap_table:
            cap.append({'investor': i.name, 'stake': i.total_shares()/self.shares * 100})
        return cap

    def find_investor(self, name):
        for i in self.cap_table:
            if name == i.name:
                return i
        return None

    def add_investor(self, name):
        new_investor = Investor(name)
        self.cap_table.append(new_investor)
        return new_investor

    def get_create_investor(self, name):
        investor = self.find_investor(name)
        if investor is None:
            investor = self.add_investor(name)
        return investor

    def new_round(self, investors, deal, covenant=None):
        
        self.round +=1

        post_money_valuation = deal['investment'] / (deal['stake']/100)
        pre_money_valuation = post_money_valuation - deal['investment']
        
        new_price = pre_money_valuation / self.shares
        new_shares = deal['investment'] / new_price

        self.price = new_price
        self.shares += new_shares
        self.equity = self.price * self.shares

        for i in investors:
            new_investor = self.get_create_investor(i['name'])
            new_investor.new_investment(self, deal['investment'] * (i['stake']/100),self.round, covenant=covenant)
            
        return None



In [77]:
capital = 1e4
price_per_share = 0.01

company = Company('consorciei', capital, capital/price_per_share)

In [78]:
cap = company.current_stake()
cap

[{'investor': 'founder', 'stake': 100.0}]

In [79]:
investors = [{'name': 'Ze', 'stake': 100}]
deal = {'investment': 100000, 'stake': 10}
company.new_round(investors, deal, covenant=None)

In [80]:
cap = company.current_stake()
cap

[{'investor': 'founder', 'stake': 90.00000000000001},
 {'investor': 'Ze', 'stake': 10.0}]

In [81]:
company.equity

999999.9999999999